In [30]:
import pandas as pd
import numpy as np
import faiss
import os

In [8]:
df = pd.read_pickle("./hnsw_all_cluster.pkl")

In [9]:
df.head(1)

,image_name,image_dir,results,image_features,cluster_id
0,g1327_czcqbl6.jpg,/nethome/kravicha3/aryan/project/dataset/Reddi...,"[1, 8887, 1415, 2028, 8885, 9459, 4633, 8590, ...","[[0.7295329, 0.30392087, 0.7336758, 0.41025618...",151


In [52]:
# features from the dataframe
df = pd.read_pickle('hnsw_all_image_results.pkl')
features = np.array(df['image_features'].to_list())
features = features.reshape(features.shape[0], features.shape[2])
# cluster training
ncentroids = 185
niter = 50
verbose = True
d = features.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(features)


Clustering 10126 points in 2048D to 185 clusters, redo 1 times, 50 iterations
  Preprocessing in 0.01 s


353476.96875

In [53]:
index_cluster = faiss.IndexFlatL2 (d)
index_cluster.add(features)
D, I = index_cluster.search(kmeans.centroids, 100)

In [54]:
I.shape

(185, 100)

In [55]:
centroid_labels = []
for centroid_id in range(I.shape[0]):
    label = {}
    for label_id in I[centroid_id]:
        thread_dir = df.iloc[label_id]['image_dir']
        thread_dir = os.path.split(thread_dir)[1]
        if thread_dir in label:
            label[thread_dir] += 1
        else:
            label[thread_dir] = 1
    centroid_labels.append(label)

In [56]:
len(centroid_labels)

185

In [57]:
centroid_num_threads = []
for centroid_id in range(len(centroid_labels)):
    max_n = 0
    max_l = ""
    for i in centroid_labels[centroid_id]:
        num = centroid_labels[centroid_id][i]
        if num>max_n:
            max_n = num
            max_l = i
    centroid_num_threads.append((max_n, max_l))

In [58]:
centroid_num_threads

[(27, '_Streaker_stealing_the_show_at_a_Tennis_game'),
 (3, '_Man_wearing_one_hundred_beanies'),
 (24, '_A_selfie_from_the_driver_of_the_Pope'),
 (16, '_Cat_Sitting_at_a_Desk_in_School'),
 (25, '_Sep_Blatter_after_a_protester_threw_money_at_him'),
 (34, '_Foggy_day'),
 (29, '_These_squirrels_fighting'),
 (9, '_Headless_hockey_player_and_terrified_referee'),
 (21, '_Dog_wearing_a_mask_at_a_dino_park'),
 (14, '_You_Catch_Your_Roommate_With_His_Pants_Down'),
 (21, '_Man_driving_his_Tricer-a-car'),
 (17, '_Group_of_Shaved_Llamas'),
 (29, '_These_piles_of_spice_at_a_market_in_Marrakesh'),
 (24, '_LeBron_James_holding_a_trophy'),
 (33, '_Man_observing_an_oil_painting_of_himself'),
 (17, '_Headless_hockey_player_and_terrified_referee'),
 (10, '_Dog_wearing_a_mask_at_a_dino_park'),
 (18, '_Shocked_Classic_Goofy'),
 (34, '_Tiger_standing_next_to_his_keeper'),
 (17, '_Adam_Savage_after_his_morning_workout'),
 (22, '_This_surprised_racoon'),
 (30, "_North_Korea's_New_Untouched_Photo_of_Kim_Jong-U

### Recall